This trains the overall classifier

In [2]:
leg_corpus_name = 'NHlegtext09_20.corpus'
overall_data_name = 'NHLegOverallData09_20.csv'
win_state_list_name = 'NHWinList09_20.list'

In [3]:
import pickle
import numpy as np
from gensim.models import Doc2Vec as d2v
from gensim.models.doc2vec import TaggedDocument as td
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

from pprint import pprint
from time import time
import logging

import warnings
from sklearn.metrics import classification_report
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore", ConvergenceWarning)

In [4]:
#load all of our documents
with open(leg_corpus_name,'rb') as f:
    legcorpus = pickle.load(f)
    
leg_data = pd.read_csv(overall_data_name)

with open(win_state_list_name,'rb') as f:
    win_state = pickle.load(f)

In [5]:
combined_docs = {}
total = 0
passed = 0
leg_target = {}

#for each document, check if it is approved, record that, then combine the list into a string
for doc in legcorpus:
    pass_flag = 0
    if doc[0] not in combined_docs:
        total += 1
        combined_docs[doc[0]] = " ".join(doc[2])
#         if total%1000 == 0:
#             print(total)

#for now, ignoring the amendments, but it seems like it could do better with the amendments
#print(list(combined_docs.keys())[0:10])

In [6]:
leg_targ = pd.DataFrame({"every stage passed":win_state}) 

bill_order = []
bill_target = []
bill_texts = []

leg_target = {}

for i, bill_id in enumerate(leg_data["bill_id"]):
    if str(bill_id) not in bill_order:
        bill_order.append(str(bill_id))
        bill_target.append(leg_targ["every stage passed"][i])
        leg_target[str(bill_id)] = leg_targ["every stage passed"][i]
        
        if bill_id in combined_docs:
            bill_texts.append(combined_docs[bill_id])
        else:
            bill_texts.append("")

In [7]:
features = leg_data
#we do need to assemble all of the data to train the model, and that is best done as close to the training as possible
X = pd.DataFrame(features)
X['leg_targ'] = leg_targ

X = X.drop_duplicates(subset=['title']) #should probably be bill_id
X = X.set_index('bill_id')
X = X.drop(columns=['time_t', 'event'])

bill_order = []
bill_target = []
bill_texts = []

count = 0
for bill_id in X.index:
    if bill_id in combined_docs:
        bill_texts.append(combined_docs[bill_id])
    else:
        bill_texts.append("")
        
X['texts'] = bill_texts
X = X.astype({'year': 'int64'}).sort_values('year', ascending=True)
X['month_introduced'] = X['month_introduced'].astype(str)

In [8]:
X_new = X[['sponsor_vec','bill_type', 'proposed_chamber',
           'primary_name','primary_party','number_sponsors',
           'committee_introduced','month_introduced', 
           #these features could be useful with more data
           #'primary_leadership','primary_ideology','public_sentiment','press',
          ]].copy()

#this dataset is slightly different than the original
amount_train = 7330
amount_test = 1000

X_dum = pd.get_dummies(X_new)

#depending on the format, you can use 
X_train_p = X_dum.iloc[0:amount_train]#.iloc
y_train = X['leg_targ'].iloc[0:amount_train]#pd.get_dummies()
X_test_p = X_dum.iloc[amount_train:amount_train+amount_test]#.iloc
y_test = X['leg_targ'].iloc[amount_train:amount_train+amount_test]#pd.get_dummies()

k_select = SelectKBest(f_classif, k=100)
X_train = k_select.fit_transform(X_train_p, y_train)
X_test = k_select.transform(X_test_p)

indices = k_select.get_support(indices=True)

print(sum(y_train)/len(y_train))
print(sum(y_test)/len(y_test))

0.352387448840382
0.257


/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 10  16  18  23  32  36  40  71  76  87  88  90  91  92 100 101 102 110
 118 119 134 144 146 152 154 161 165 168 187 198 203 218 232 233 235 237
 243 245 252 253 255 264 265 275 276 279 283 299 307 313 319 325 331 334
 336 353 367 384 387 390 404 406 416 417 421 434 454 463 485 489 492 502
 506 526 531 536 543 544 547 553 566 569 571 577 585 589 612 631 637 639
 640 642 674 677 688 694 704 709 716 722 725 726 727 728 731 738 743 747
 757 778 782] are constant.
  UserWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [9]:
compressed_legtext = pd.DataFrame(X['texts'].copy())
#prepare the documents by using tfidf vectors:

#from https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a
stemmer = SnowballStemmer("english", ignore_stopwords=True)
#adjust this stemmer so that it does more than one word in the tokens

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

stemmed_count_vect = StemmedCountVectorizer(stop_words='english', ngram_range=(1,4))
stem_counts = stemmed_count_vect.fit_transform(compressed_legtext['texts'][0:amount_train])
stem_counts_test = stemmed_count_vect.transform(compressed_legtext['texts'][amount_train:amount_train+amount_test])

tfidf = TfidfTransformer()
tfidf_vectors = tfidf.fit_transform(stem_counts)
tfidf_vectors_test = tfidf.transform(stem_counts_test)

In [10]:
#do the final pick after, since we only want to run the above cell once
k_best_tfidf = SelectKBest(f_classif, k=1000)
final_tfidf_features = k_best_tfidf.fit_transform(tfidf_vectors, y_train)
final_tfidf_features_test = k_best_tfidf.transform(tfidf_vectors_test)
word_indices = k_best_tfidf.get_support(indices=True)

In [11]:
#combine tfidf legtext and the other features:
legtext_train = final_tfidf_features.todense()

legtext_test = final_tfidf_features_test.todense()

X_train = np.concatenate([np.array(X_train),legtext_train], axis=1)
X_test = np.concatenate([np.array(X_test),legtext_test], axis=1)

In [12]:
means = []
n = 1
for i in range(n):
    clf = MLPClassifier(hidden_layer_sizes=(50,),
                        activation='relu',
                        random_state=i+100, max_iter=50,
                        solver='lbfgs', alpha=0.01,
                        tol=0.001, n_iter_no_change=4)
    #The MNB classifier can be useful too
    #clf = MultinomialNB(alpha=0)#alpha=0.001,DecisionTreeClassifier()#MultinomialNB()##19 ##
    
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    predicted_prob = clf.predict_proba(X_test)
    new_mean = np.mean(predicted == y_test)
    means.append(new_mean)
    
    if n > 1:
        print(new_mean)
    
mean=np.mean(np.array(means))
print("overall mean:", mean)
print(classification_report(y_test, predicted, digits=4))


overall mean: 0.959
              precision    recall  f1-score   support

           0     0.9743    0.9704    0.9724       743
           1     0.9154    0.9261    0.9207       257

    accuracy                         0.9590      1000
   macro avg     0.9449    0.9482    0.9465      1000
weighted avg     0.9592    0.9590    0.9591      1000



In [18]:
def get_actions(action_clf, data, desired_probability, epochs=10, alpha=0.1, epsilon=0.0001,
                first_k = 100, labels=None, verbose=False): 
    #In the future, include a feature_mask instead of first_k
    #This returns all actions. May want to create something that does top-n actions.
    """
    action_clf: a classifier that is compatible with sklearn, typically the overall classifier
    data: a single row of bill data (numpy array)
    desired_probability: an float between 0 and 1 inclusive
    first_k: only act on (and print) the first k actions
    labels: the named labels of the inputs
    base_labels: stemmed labels (i.e. not split into pd dummies)
    epochs: number of epochs to run iterative gradient descent
    alpha: step size to move in the direction of the gradient
    epsilon: the size of the perturbation of the inputs
    verbose: if True, will print predicted probability at each epoch, and print final actions
    """
    
    initial_prob = action_clf.predict_proba(data)
    
    if verbose:
        print("Initial probability of passing:", initial_prob)
    
    DZDY = desired_probability - initial_prob #gradient of the loss wrt the output. Scalar for a single input
    DZDX = np.zeros_like(data) #empty array of the same size as data (i.e. x)
    DZDX_c = np.zeros_like(data)
    
    x_p = data.copy()
    x_n = data.copy()

    for epoch in range(epochs):
        #can handle batches of data (multiple bills):
        for t, row in enumerate(x_p): 
            #since our dataset is metadata followed by tfidf words, and we can't really affect the words yet:
            for i in range(first_k): 
                #perturb the inputs a little bit
                x_p[t][i] += epsilon
                x_n[t][i] -= epsilon
                #predict from the perturbed input
                f_p = action_clf.predict_proba(x_p)
                f_n = action_clf.predict_proba(x_n)
                #this is basically newtons method to find the gradient
                DZDX[t][i] = np.sum(DZDY*((f_p-f_n)/(2*epsilon)))
                # reset the tensors to their original state 
                x_p[t][i] -= epsilon
                x_n[t][i] += epsilon
                
        DZDX_c += DZDX
        adjusted_data = data + DZDX_c
        adjusted_data[adjusted_data < 0] = 0
        DZDX_c = adjusted_data - data
        
        new_prob = action_clf.predict_proba(data + DZDX_c)
        
        if verbose:
            print("Prob on epoch", epoch, ":", new_prob)
            
        DZDY = desired_probability - new_prob
    
    if verbose:
        print("Final actions:")
        for i in range(first_k):
            print("{0:.5f}".format(DZDX_c[0][i]), "\t", labels[i])

    return DZDX_c
    
    
def get_recommendations(clf, data, desired_probability, DZDX_c, label_action_count, 
                        metadata_num, labels, base_labels, verbose=False):
    """
    get_recommendations basically turns get_actions into useful information
    clf: sklearn classifier, typically the overall classifier, built for a MLP (neural network)
    data: 
    desired_probability: between 0 and 1
    DZDX_c: the cumulative change in the variable vector as produced by get_actions
    metadata_num: number of selected dummy metadata features (selects the first metadata_num features)
    base_labels: the base labels that pd.dummies has made expanded into one-hot encodings
    label_action_count: an iterable of ints indicating the number of options to give for that feature. 
        Has the same length as base_labels. 
        ex: base_labels = [committee, primary_sponsor], label_action_count = [3, 10]
    """
    feature_importances = {base_label:[] for base_label in base_labels}
    for i, base_label in enumerate(base_labels):
        #this works because pd dummies puts things in order
        for j in range(metadata_num):
            if labels[j][0:len(base_label)] == base_label:
                #store in feature_importances such that they can be sorted later
                feature_importances[base_label].append((labels[j], DZDX_c[0][j]))
            
    #sort each label by absolute feature importance
    for i, label in enumerate(feature_importances):
        #sort by the absolute value of that feature importance 
        feature_importances[label].sort(key=lambda x: abs(x[1]), reverse=True)
    
    final_actions = {base_label:[] for base_label in base_labels}
    for i, count in enumerate(label_action_count):
        final_actions[base_labels[i]] = feature_importances[base_labels[i]][0:count]
        print(i)
        print(base_labels[i])
        print(final_actions[base_labels[i]])
    
    if verbose:
        for key in final_actions.keys():
            
            if final_actions[key] != []:
                print(key)
                for i in final_actions[key]:
                    print("\t", "{0:.5f}".format(i[1]), "\t\t", i[0])

    return final_actions


In [19]:
overall_clf = clf
#HB1680: bill_id is 1294143
bill_id = 1287298
HB_test = X_dum.loc[[bill_id]]#.reshape(1, -1)

HB_test = k_select.transform(HB_test)
indices = k_select.get_support(indices=True)

HB_test = np.array(HB_test)

HB_legtext_counts = stemmed_count_vect.transform(X[['texts']].loc[bill_id])
HB_legtext_tfidf = tfidf.transform(HB_legtext_counts)
HB_legtext = k_best_tfidf.transform(HB_legtext_tfidf).todense()
HB_legtext = np.array(HB_legtext)
HB_data = np.concatenate([HB_test, HB_legtext], axis=1)
#print(HB_data)

HB_prob = overall_clf.predict_proba(HB_data)

action_vector = get_actions(overall_clf, HB_data, 1, first_k = 100, labels=X_dum.columns[indices],  
                            epochs=10, alpha=0.1, epsilon=0.0001, verbose=True) #in the future, include a feature_mask instead of hard-coding features

# Order: 'sponsor_vec', 'chamber', 'bill_type', 'proposed_chamber', 'primary_name', 
# 'primary_party', 'number_sponsors','committee_introduced', 'month_introduced'

label_action_count = [1,0,1,3,5,2,1,0,3] #probably change this to a dictionary
print("Suggested actions:\n")
recs_from_action_count = get_recommendations(overall_clf, HB_data, 1, action_vector, label_action_count,
                                             100, X_dum.columns[indices], X_new.columns, verbose=True)
#base_labels=X.columns,

Initial probability of passing: [[0.985244 0.014756]]
Prob on epoch 0 : [[0.94651384 0.05348616]]
Prob on epoch 1 : [[0.83724355 0.16275645]]
Prob on epoch 2 : [[0.66923079 0.33076921]]
Prob on epoch 3 : [[0.55883787 0.44116213]]
Prob on epoch 4 : [[0.51840236 0.48159764]]
Prob on epoch 5 : [[0.50568061 0.49431939]]
Prob on epoch 6 : [[0.50175129 0.49824871]]
Prob on epoch 7 : [[0.50053984 0.49946016]]
Prob on epoch 8 : [[0.50016641 0.49983359]]
Prob on epoch 9 : [[0.5000513 0.4999487]]
Final actions:
0.02618 	 sponsor_vec
0.00048 	 number_sponsors
0.03519 	 bill_type_B
0.00438 	 bill_type_CA
0.00000 	 bill_type_CR
0.00000 	 bill_type_R
-0.02758 	 proposed_chamber_H
0.00554 	 proposed_chamber_S
0.00450 	 primary_name_Andrew Hosmer
0.00000 	 primary_name_Andrew Manuse
0.01517 	 primary_name_Betsey Patten
0.01476 	 primary_name_Betsi DeVries
0.02615 	 primary_name_Bob Odell
0.00831 	 primary_name_Brian Rhodes
0.00000 	 primary_name_Brian Stone
0.00000 	 primary_name_Carol McGuire
0.00000

IndexError: index 8 is out of bounds for axis 0 with size 8